In [2]:
import pandas as pd
import numpy as np
import ast
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies=movies.merge(credits,on='title')
movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.dropna(inplace=True)
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)
def convert_cast(obj):
    L=[]
    count=0
    for i in ast.literal_eval(obj):
        if count<3:
            L.append(i['name'])
            count += 1
        else:
            break
    return L
movies['cast']=movies['cast'].apply(convert_cast)
def fetch_director(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
        break
    return L
movies['crew']=movies['crew'].apply(fetch_director)
movies['overview']=movies['overview'].apply(lambda x:x.split())
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
new_df=movies[['movie_id','title','tags']]
new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
def recommend(movie):
    for i in range(len(new_df)):
        if new_df.iloc[i].title==movie:
            movie_index=i
            break
    distances = similarity[movie_index]
    similar_movies = []
    for i in range(len(distances)):
        similar_movies.append([i,distances[i]])
    similar_movies.sort(key=lambda x:x[1],reverse=True)
    for i in range(1,6):
        index = similar_movies[i][0]
        print(new_df.iloc[index].title)
import pickle
pickle.dump(new_df,open('movies.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))
print("Saving movies and similarity...")
pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))
print("Done.")





C:\Users\Akshat Paliwal\AppData\Local\Temp\ipykernel_27040\3823709190.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\Akshat Paliwal\AppData\Local\Temp\ipykernel_27040\3823709190.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


Saving movies and similarity...
Done.
